In [45]:
import sympy
import numpy as np
from dataclasses import dataclass
from typing import List

In [37]:
Point = sympy.Function('Point')
Line = sympy.Function('Line')
Extrude = sympy.Function('Extrude')

In [43]:
a = Point(0,0)
b = Point(1,1)
l = Line(a, b)
rect = Extrude(l, 1)

In [44]:
rect

Extrude(Line(Point(0, 0), Point(1, 1)), 1)

In [128]:
num = float # int

def box_union(*boxes):
    points = np.concatenate(boxes, 0)
    return np.stack([points.min(axis=0), points.max(axis=0)])

class Shape:
    pass

@dataclass
class Point(Shape):
    x: num
    y: num

    def eval(self):
        return f'<circle cx="10" cy="10" r="2" fill="green"/>'

    def box(self):
        return np.array([[self.x,self.y],[self.x,self.y]])

@dataclass
class Line(Shape):
    start: Point
    end: Point

    def eval(self):
        return f'<line x1="{self.start.x}" y1="{self.start.y}" x2="{self.end.x}" y2="{self.end.y}" stroke-width="1" stroke="red"/>'
    
    def length(self):
        return sqrt((self.end.x - self.start.x)**2 + (self.end.y - self.start.y)**2)

    def box(self):
        return box_union(self.start.box(), self.end.box())


@dataclass
class Extrude(Shape):
    line: Line
    distance: num

    def poly(self):
        norm = self.line.length()
        perp_x = self.distance * (self.line.start.x - self.line.end.x) / norm
        perp_y = self.distance * (self.line.end.x - self.line.start.x) / norm
        
        rect = Polygon([
           self.line.start,
           self.line.end,
           Point(self.line.end.x + perp_x, self.line.end.y + perp_y),
           Point(self.line.start.x + perp_x, self.line.start.y + perp_y)
        ])

        return rect

    def eval(self):
        return self.poly().eval()

    def box(self):
        return self.poly().box()

@dataclass
class Polygon(Shape):
    points: List[Point]

    def eval(self):
        points_str = " ".join(f'{p.x},{p.y}' for p in self.points)
        return f'<polygon points="{points_str}" fill="blue" stroke="none"/>'

    def box(self):
        return box_union(*[p.box() for p in self.points])

a = Point(0,0)
b = Point(1,1)
l = Line(a,b)
rect = Extrude(l, 1)
rect.eval()

@dataclass
class ShapeList(Shape):
    shapes: List[Shape]

    def box(self):
        return box_union(*[s.box() for s in self.shapes])
    
    def eval(self):
        contents_svg = '\n'.join([s.eval() for s in self.shapes])
        return f'<g>{contents_svg}</g>'

@dataclass
class Translate(Shape):
    x: num
    y: num
    shape: Shape

    def box(self):
        return self.shape.box + np.array([self.x, self.y])

    def eval(self):
        return f'<g transform="translate({self.x} {self.x})>{self.shape.eval()}</g>'

def draw(shape):
    shape_svg = shape.eval()
    box = shape.box()
    min_corner = box[0]
    box_size = box[1] - box[0]
    view_box = ' '.join([' '.join(str(x) for x in min_corner), ' '.join(str(x) for x in box_size)])

    return f'<svg viewBox="{view_box}" xmlns="http://www.w3.org/2000/svg">{shape_svg}</svg>'

In [129]:
rect_svg = draw(rect)
SVG(data=rect_svg)

In [125]:
rect_svg[210:220]

'5475", fil'

In [34]:
from IPython.display import SVG

In [119]:
svg_example = '''<svg viewBox="-4.0 -4.0 20.0 10.0" xmlns="http://www.w3.org/2000/svg">
  <!-- Example of a polygon with the default fill -->
  <polygon points="0.0,10.0 5.0,2.5 5.0,7.5 10.0,0.0" />
</svg>'''
SVG(data=svg_example)

In [2]:
# numbers
# add
# line
# extrude

In [29]:
class Line:
    def __init__(self, x1, y1, x2, y2):
        self.x1 = x1
        self.y1 = y1
        self.x2 = x2
        self.y2 = y2
    def __call__(self):
        return f'line({self.x1()}, {self.y1()}, {self.x2()}, {self.x2()})'

class Number:
    def __init__(self, n):
        self.n = n
    def __call__(self):
        return self.n

from math import sqrt

class Polygon:
    def __init__(self, *points):
        self.points = points
    
    def __call__(self):
        points_str = " ".join(f'{str(a)},{str(b)}' for (a,b) in self.points)
        return f'<polygon points="{points_str}"/>'

class Extrude:
    def __init__(self, l, d):
        self.l = l
        self.d = d
    def __call__(self):
        x1 = self.l.x1()
        x2 = self.l.x2()
        y1 = self.l.y1()
        y2 = self.l.y2()

        perp_y = (y1 - y2)
        perp_x = (x2 - x1)
        perp_norm = sqrt(perp_y**2 + perp_x**2)
        n_perp_y = perp_y / perp_norm
        n_perp_x = perp_x / perp_norm

        return Polygon(
            (x1,y1),
            (x2,y2), 
            (x1+n_perp_x*self.d(), y1+n_perp_y*self.d()),
            (x2+n_perp_x*self.d(), y2+n_perp_y*self.d())
        )()

In [30]:
rect = Extrude(Line(Number(0),Number(0), Number(1), Number(2)), Number(1))

In [31]:
rect()

'<polygon points="0,0 1,2 0.4472135954999579,-0.8944271909999159 1.4472135954999579,1.1055728090000843"/>'

In [ ]:
# We want to go from a program text to an lambda plus a set of parameters. The lambda should generate an SVG using those parameters
